pip install torch transformers sklearn pandas

For Mac M1:

curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh

In [22]:
from transformers import pipeline, set_seed
generator = pipeline('text-generation', model='gpt2')
set_seed(42)
generator("Hello, I'm Elon Musk,", max_length=30, num_return_sequences=5)

loading configuration file https://huggingface.co/gpt2/resolve/main/config.json from cache at /Users/yu/.cache/huggingface/transformers/fc674cd6907b4c9e933cb42d67662436b89fa9540a1f40d7c919d0109289ad01.7d2e0efa5ca20cef4fb199382111e9d3ad96fd77b849e1d4bed13a66e1336f51
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params"

[{'generated_text': "Hello, I'm Elon Musk, I'm CEO of Intel. In my previous years working at Intel, I spent many years on the research and development"},
 {'generated_text': "Hello, I'm Elon Musk, and I'm President/CEO of Tesla. And it's so humbling. This is the kind of kind of"},
 {'generated_text': "Hello, I'm Elon Musk, the world's most valuable entrepreneur, and I wanted to hear from you about why I'm passionate about building a network"},
 {'generated_text': "Hello, I'm Elon Musk, a billionaire tech investor and investor in SpaceX and the Space Exploration Technologies Corporation. If you aren't familiar he is Tesla"},
 {'generated_text': "Hello, I'm Elon Musk, the CEO and Co-Founder of SpaceX and founder of the SpaceX Education Lab. It's not easy, but"}]

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_csv("dataset/train_cleaned.csv")['content'].to_numpy()
train, test = train_test_split(data,test_size=0.15)
traindata = ''
testdata = ''
for i in train:
    traindata += i +'\n'
f = open('train_dataset.txt','w')
f.write(traindata)
for i in test:
    testdata += i +'\n'
f = open('test_dataset.txt','w')
f.write(testdata)

172967

In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")
 
train_path = 'train_dataset.txt'
test_path = 'test_dataset.txt'

In [4]:
from transformers import TextDataset,DataCollatorForLanguageModeling

def load_dataset(train_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=64)

    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=64)

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator

train_dataset,test_dataset,data_collator = load_dataset(train_path,test_path,tokenizer)

/opt/anaconda3/envs/DL/lib/python3.9/site-packages/transformers/data/datasets/language_modeling.py:54: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [23]:
from transformers import Trainer, TrainingArguments, AutoModelWithLMHead

model = AutoModelWithLMHead.from_pretrained("gpt2")

training_args = TrainingArguments(
    output_dir="./gpt2-musk", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=3, # number of training epochs
    per_device_train_batch_size=32, # batch size for training
    per_device_eval_batch_size=64,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    )

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    #prediction_loss_only=True,
)

loading configuration file https://huggingface.co/gpt2/resolve/main/config.json from cache at /Users/yu/.cache/huggingface/transformers/fc674cd6907b4c9e933cb42d67662436b89fa9540a1f40d7c919d0109289ad01.7d2e0efa5ca20cef4fb199382111e9d3ad96fd77b849e1d4bed13a66e1336f51
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params"

In [6]:
trainer.train()

/opt/anaconda3/envs/DL/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 3601
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 339
100%|██████████| 339/339 [15:28<00:00,  2.23s/it]

Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 339/339 [15:28<00:00,  2.74s/it]

{'train_runtime': 928.0293, 'train_samples_per_second': 11.641, 'train_steps_per_second': 0.365, 'train_loss': 4.626492930724558, 'epoch': 3.0}


TrainOutput(global_step=339, training_loss=4.626492930724558, metrics={'train_runtime': 928.0293, 'train_samples_per_second': 11.641, 'train_steps_per_second': 0.365, 'train_loss': 4.626492930724558, 'epoch': 3.0})

In [7]:
trainer.save_model()

Saving model checkpoint to ./gpt2-musk
Configuration saved in ./gpt2-musk/config.json
Model weights saved in ./gpt2-musk/pytorch_model.bin


In [16]:
from transformers import pipeline

tweet = pipeline('text-generation',model='gpt2-musk', tokenizer=tokenizer )

loading configuration file gpt2-musk/config.json
Model config GPT2Config {
  "_name_or_path": "gpt2-musk",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "torch_dtype": "float32",
  "transformers_version": "4.19.4",
  "use_cache": true,
  "vocab_size": 50257
}

loading config

In [21]:
#generator = pipeline('text-generation', model='gpt2')
set_seed(42)
tweet("Hello, I'm Elon Musk,", max_length=50, num_return_sequences=5)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "Hello, I'm Elon Musk, aka Tesla CEO. For the most part, this is about doing the right thing. I think the Tesla owner will be happy to take it, but will want to steer clear of me. Tesla owners should take it"},
 {'generated_text': "Hello, I'm Elon Musk, and I am President/CEO of Tesla.\nHaha, actually I was thinking about this! Tesla Roadster was literally amazing during construction of my first electric car. Will be amazing!\nThere will be many"},
 {'generated_text': "Hello, I'm Elon Musk, first employee to date for SpaceX, is a rocket engineer.\nSure!\nThere are other places on Earth that have all the oxygen that we need to work. I will check that out in a few weeks."},
 {'generated_text': "Hello, I'm Elon Musk, a former tech investor &amp; founder of SpaceX\nThere are probably some cool cars coming to the Tesla Model S team.\nAlso, Tesla makes cars in Canada/US. Also, the US version is already"},
 {'generated_text': "Hello, I'm Elon Musk, creator &amp; owner of S